In [442]:
import numpy as np
import pandas as pd
import pickle
import codecs, json

# you will get error because i did not add book data if you are cloning from github because that was to much 
# But you can download from kagle by searching book recomendation data

In [260]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')

C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [261]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [262]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [263]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [264]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [265]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [266]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [267]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [268]:
books['ISBN'].duplicated().sum()

0

In [269]:
ratings.duplicated().sum()

0

In [270]:
users.duplicated().sum()


0

## Popularity Based Recommender System

In [271]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [272]:
ratings_with_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [273]:
ratings_with_name.groupby('Book-Title').count()

,User-ID,ISBN,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
Book-Title,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",4,4,4,4,4,4,4,4,4
Always Have Popsicles,1,1,1,1,1,1,1,1,1
Apple Magic (The Collector's series),1,1,1,1,1,1,1,1,1
"Ask Lily (Young Women of Faith: Lily Series, Book 5)",1,1,1,1,1,1,1,1,1
Beyond IBM: Leadership Marketing and Finance for the 1990s,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
Ã?Â?lpiraten.,2,2,2,2,2,2,2,2,2
Ã?Â?rger mit Produkt X. Roman.,4,4,4,4,4,4,4,4,4
Ã?Â?sterlich leben.,1,1,1,1,1,1,1,1,1


In [282]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating']
num_rating_df = num_rating_df.reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)

In [283]:
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [284]:
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating']
avg_rating_df = avg_rating_df.reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_ratings'},inplace=True)

In [285]:
avg_rating_df

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [286]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [287]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings', ascending=False).head(50)

In [288]:
popular_df.merge(books,on='Book-Title')

,Book-Title,num_ratings,avg_ratings,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,0439136350,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...
1,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,0439136369,J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...
2,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804,0786222743,J. K. Rowling,2000,Thorndike Press,http://images.amazon.com/images/P/0786222743.0...,http://images.amazon.com/images/P/0786222743.0...,http://images.amazon.com/images/P/0786222743.0...
3,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,0439139597,J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...
4,Harry Potter and the Goblet of Fire (Book 4),387,5.824289,0439139600,J. K. Rowling,2002,Scholastic Paperbacks,http://images.amazon.com/images/P/0439139600.0...,http://images.amazon.com/images/P/0439139600.0...,http://images.amazon.com/images/P/0439139600.0...
...,...,...,...,...,...,...,...,...,...,...
191,Lord of the Flies,259,3.818533,0698102193,William G. Golding,1978,Putnam Pub Group,http://images.amazon.com/images/P/0698102193.0...,http://images.amazon.com/images/P/0698102193.0...,http://images.amazon.com/images/P/0698102193.0...
192,A Prayer for Owen Meany,413,3.796610,0345361792,John Irving,1990,Ballantine Books,http://images.amazon.com/images/P/0345361792.0...,http://images.amazon.com/images/P/0345361792.0...,http://images.amazon.com/images/P/0345361792.0...
193,A Prayer for Owen Meany,413,3.796610,0345363523,John Irving,1989,Ballantine Books,http://images.amazon.com/images/P/0345363523.0...,http://images.amazon.com/images/P/0345363523.0...,http://images.amazon.com/images/P/0345363523.0...
194,"The Vampire Lestat (Vampire Chronicles, Book II)",301,3.777409,0345313860,ANNE RICE,1986,Ballantine Books,http://images.amazon.com/images/P/0345313860.0...,http://images.amazon.com/images/P/0345313860.0...,http://images.amazon.com/images/P/0345313860.0...


In [289]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_ratings']]

In [290]:
popular_df

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [291]:
top50 = popular_df['num_ratings'].tolist()

In [446]:
top50

[428,
 387,
 278,
 347,
 556,
 281,
 368,
 575,
 260,
 510,
 898,
 430,
 449,
 1295,
 284,
 253,
 265,
 774,
 283,
 493,
 723,
 268,
 335,
 252,
 409,
 526,
 283,
 585,
 275,
 664,
 389,
 265,
 279,
 490,
 321,
 302,
 322,
 318,
 336,
 314,
 256,
 262,
 278,
 266,
 269,
 457,
 259,
 413,
 301,
 356]

In [292]:




json.dump(top50, codecs.open('C:\\Users\\Dell\\Desktop\\Project\\top50\\numRatings.json', 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Dell\\Desktop\\Project\\top50\\numRatings.json'

## Collaborative Filtering Based Recommender System

In [383]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index
padhe_likhe_users

Int64Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,   6543,
            ...
            271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427, 277639, 278418], dtype='int64', name='User-ID', length=811)

In [384]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [385]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index

In [386]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [387]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [388]:
pt.fillna(0,inplace=True)

In [318]:
pt

User-ID,254,...
Book-Title,,
10 Lb. Penalty,0.0,...
16 Lighthouse Road,0.0,...
1984,9.0,...
1st to Die: A Novel,0.0,...
2010: Odyssey Two,0.0,...
204 Rosewood Lane,0.0,...
24 Hours,0.0,...
2nd Chance,0.0,...
3rd Degree,0.0,...


In [328]:
hello = pt.merge(books,on='Book-Title')


In [300]:
hello = hello.drop_duplicates('Book-Title')

In [301]:
hello = hello[['Book-Title','Book-Author','Year-Of-Publication','Image-URL-M']].set_index('Book-Title')

In [201]:
hello

,Book-Author,Year-Of-Publication,Image-URL-M
Book-Title,,,
'Salem's Lot,Stephen King,1990,http://images.amazon.com/images/P/0451168089.0...
10 Lb. Penalty,Dick Francis,1997,http://images.amazon.com/images/P/0399143025.0...
101 Dalmatians,Justine Korman,1996,http://images.amazon.com/images/P/0307001164.0...
16 Lighthouse Road,Debbie Macomber,2001,http://images.amazon.com/images/P/1551668300.0...
1984,George Orwell,1990,http://images.amazon.com/images/P/0451524934.0...
...,...,...,...
Zlata's Diary: A Child's Life in Sarajevo,Zlata Filipovic,0,http://images.amazon.com/images/P/0590487922.0...
Zoya,Danielle Steel,1989,http://images.amazon.com/images/P/0440203856.0...
"\O\"" Is for Outlaw""",SUE GRAFTON,2001,http://images.amazon.com/images/P/0449003787.0...


In [92]:
helll = pd.read_csv('pop.csv')

###### helll

In [248]:
pt

User-ID,243,254,507,638,643,741,882,929,1211,1424,1435,1674,1733,1848,1903,2030,2033,2110,2132,2179,2276,2358,2766,2891,2977,3145,3167,3346,3363,3371,3373,3556,3675,3757,3827,4017,4092,4157,4385,4622,4938,5209,5439,5539,5543,5582,5609,5903,6073,6242,6251,6323,6431,6543,6563,6575,6772,7125,7128,7158,7210,7283,7286,7346,7569,7887,7915,7958,8019,8066,8067,8167,8245,8253,8284,8362,8454,8487,8529,8681,8734,8782,8839,8890,8936,8971,9141,9227,9417,9613,9747,9856,9908,10030,10067,10118,10314,10354,10447,10560,10819,11120,11224,11245,11400,11601,11629,11676,11687,11724,11993,12100,12272,12538,12824,12835,12982,13082,13093,13273,13540,13552,13582,13600,13850,13935,13995,14079,14232,14326,14387,14422,14449,14456,14521,14638,14698,14768,14897,14966,15021,15049,15208,15408,15418,15497,15589,15651,15670,15819,15945,15957,16106,16155,16161,16413,16488,16599,16601,16634,16718,16721,16795,16916,16943,16966,16996,17003,17190,17282,17507,17667,17859,17950,18067,18082,18172,18384,18401,18995,19085,19493,19522,19664,19681,19711,19842,20060,20115,20119,20172,20180,20201,20409,20462,20627,20859,20971,21011,21014,21031,21229,21252,21356,21364,21404,21484,21576,21618,21659,22074,22365,22495,22521,22625,22818,23288,23511,23571,23680,23768,23871,23872,23902,24186,24194,24433,24539,24544,24641,24733,24921,24933,24934,24941,24945,25131,25311,25409,25436,25533,25601,25630,25695,25804,25981,26084,26240,26443,26516,...,254299,254377,254405,254465,254467,254502,254543,254676,254822,254899,254971,255078,255092,255218,255489,255546,255618,255719,255814,255846,255979,256072,256167,256247,256402,256407,256843,256844,257028,257198,257204,257327,257368,257450,257493,257700,257962,258011,258084,258152,258185,258240,258270,258352,258482,258534,258690,258884,258913,258938,259004,259028,259035,259057,259114,259118,259155,259249,259260,259264,259378,259380,259625,259626,259629,259660,259829,259901,259930,260183,260662,260699,260897,260944,261037,261105,261603,261829,261998,262186,262310,262399,262459,262902,262974,262998,263107,263163,263325,263344,263383,263460,263645,263877,264031,264082,264152,264311,264317,264321,264355,264525,264543,264637,264657,264903,264947,264996,265083,265115,265313,265403,265487,265553,265591,265595,265607,265834,265889,266056,266226,266229,266283,266522,266650,266753,266807,266840,266865,266866,267033,267061,267249,267282,267409,267510,267538,267635,267788,267830,268022,268030,268032,268110,268300,268330,268622,268657,268691,268711,268751,268833,268932,269157,269321,269439,269557,269566,269719,269728,269738,269799,269835,269841,269890,270256,270352,270554,270605,270629,270713,270820,270838,270906,271144,271176,271195,271245,271284,271375,271448,271538,271558,271622,271705,272077,272120,272225,272482,272573,272626,272640,272715,272897,273086,273110,273113,273190,273718,273906,273979,273993,274004,274056,274061,274111,274301,274308,274325,274423,274466,274549,274634,274656,274808,275020,275239,275276,275281,275922,275970,276018,276050,276165,276231,276307,276463,276641,276670,276680,276688,276925,276929,277157,277195,277203,277378,277427,277478,277639,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Book-Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [249]:
pd.set_option('display.expand_frame_repr',False)

In [252]:
pt

User-ID,243,254,507,638,643,741,882,929,1211,1424,1435,1674,1733,1848,1903,2030,2033,2110,2132,2179,2276,2358,2766,2891,2977,3145,3167,3346,3363,3371,3373,3556,3675,3757,3827,4017,4092,4157,4385,4622,4938,5209,5439,5539,5543,5582,5609,5903,6073,6242,6251,6323,6431,6543,6563,6575,6772,7125,7128,7158,7210,7283,7286,7346,7569,7887,7915,7958,8019,8066,8067,8167,8245,8253,8284,8362,8454,8487,8529,8681,8734,8782,8839,8890,8936,8971,9141,9227,9417,9613,9747,9856,9908,10030,10067,10118,10314,10354,10447,10560,10819,11120,11224,11245,11400,11601,11629,11676,11687,11724,11993,12100,12272,12538,12824,12835,12982,13082,13093,13273,13540,13552,13582,13600,13850,13935,13995,14079,14232,14326,14387,14422,14449,14456,14521,14638,14698,14768,14897,14966,15021,15049,15208,15408,15418,15497,15589,15651,15670,15819,15945,15957,16106,16155,16161,16413,16488,16599,16601,16634,16718,16721,16795,16916,16943,16966,16996,17003,17190,17282,17507,17667,17859,17950,18067,18082,18172,18384,18401,18995,19085,19493,19522,19664,19681,19711,19842,20060,20115,20119,20172,20180,20201,20409,20462,20627,20859,20971,21011,21014,21031,21229,21252,21356,21364,21404,21484,21576,21618,21659,22074,22365,22495,22521,22625,22818,23288,23511,23571,23680,23768,23871,23872,23902,24186,24194,24433,24539,24544,24641,24733,24921,24933,24934,24941,24945,25131,25311,25409,25436,25533,25601,25630,25695,25804,25981,26084,26240,26443,26516,...,254299,254377,254405,254465,254467,254502,254543,254676,254822,254899,254971,255078,255092,255218,255489,255546,255618,255719,255814,255846,255979,256072,256167,256247,256402,256407,256843,256844,257028,257198,257204,257327,257368,257450,257493,257700,257962,258011,258084,258152,258185,258240,258270,258352,258482,258534,258690,258884,258913,258938,259004,259028,259035,259057,259114,259118,259155,259249,259260,259264,259378,259380,259625,259626,259629,259660,259829,259901,259930,260183,260662,260699,260897,260944,261037,261105,261603,261829,261998,262186,262310,262399,262459,262902,262974,262998,263107,263163,263325,263344,263383,263460,263645,263877,264031,264082,264152,264311,264317,264321,264355,264525,264543,264637,264657,264903,264947,264996,265083,265115,265313,265403,265487,265553,265591,265595,265607,265834,265889,266056,266226,266229,266283,266522,266650,266753,266807,266840,266865,266866,267033,267061,267249,267282,267409,267510,267538,267635,267788,267830,268022,268030,268032,268110,268300,268330,268622,268657,268691,268711,268751,268833,268932,269157,269321,269439,269557,269566,269719,269728,269738,269799,269835,269841,269890,270256,270352,270554,270605,270629,270713,270820,270838,270906,271144,271176,271195,271245,271284,271375,271448,271538,271558,271622,271705,272077,272120,272225,272482,272573,272626,272640,272715,272897,273086,273110,273113,273190,273718,273906,273979,273993,274004,274056,274061,274111,274301,274308,274325,274423,274466,274549,274634,274656,274808,275020,275239,275276,275281,275922,275970,276018,276050,276165,276231,276307,276463,276641,276670,276680,276688,276925,276929,277157,277195,277203,277378,277427,277478,277639,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Book-Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [331]:
pt.shape

(985, 1377)

In [424]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',5)
# pd.set_option('display.width',500)

In [425]:
pt

User-ID,254,2276,...,277639,278418
Book-Title,,,,,
1984,9.0,0.0,...,0.0,0.0
1st to Die: A Novel,0.0,0.0,...,0.0,0.0
2nd Chance,0.0,10.0,...,0.0,0.0
4 Blondes,0.0,0.0,...,0.0,0.0
A Bend in the Road,0.0,0.0,...,0.0,0.0
...,...,...,...,...,...
Year of Wonders,0.0,0.0,...,0.0,0.0
You Belong To Me,0.0,0.0,...,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,...,0.0,0.0


In [389]:
from sklearn.metrics.pairwise import cosine_similarity

In [390]:
similarity_scores = cosine_similarity(pt)

In [391]:
similarity_scores

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [407]:
def recommend(book_name):
    # fetching the index
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[0:30]
#     print(index)
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
        
    return data

In [369]:
1984 fiction
1st to Die: A Novel -- crime
Night over Water -- thriller ---
Anne Frank: The Diary of a Young Girl ---physology
Interview with the Vampire --- horror
A Walk to Remember --- romance
It Was on Fire When I Lay Down on It --- self help 
The Fellowship of the Ring (The Lord of the Rings, Part 1) -- fantasy

SyntaxError: invalid syntax (<ipython-input-369-2a50e086f1e0>, line 1)

In [509]:
recommend("Night over Water")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [337]:
pt

User-ID,254,...
Book-Title,,
1984,9.0,...
1st to Die: A Novel,0.0,...
2010: Odyssey Two,0.0,...
24 Hours,0.0,...
2nd Chance,0.0,...
4 Blondes,0.0,...
84 Charing Cross Road,0.0,...
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,...
A Bend in the Road,0.0,...


In [354]:
# book_name = 'The 48 Laws of Power'
# pd.where(books.index==book_name)[0][0]
books[books['Book-Title']=='Meditations (Great Books in Philosophy)']

,ISBN,...
71218,0879757027,...


In [28]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))

NameError: name 'pt' is not defined

In [111]:
type(books)

pandas.core.frame.DataFrame

In [114]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
similarity_scores.shape

(706, 706)

In [113]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [115]:
similarity_scores

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [34]:
data = pd.read_csv('New.csv')

In [35]:
data

,Book-Title,Book-Author,Year-Of-Publication,Image-URL-M
0,1984,George Orwell,1990,http://images.amazon.com/images/P/0451524934.0...
1,1st to Die: A Novel,James Patterson,2002,http://images.amazon.com/images/P/0446610038.0...
2,2nd Chance,James Patterson,2002,http://images.amazon.com/images/P/0316693200.0...
3,4 Blondes,Candace Bushnell,2002,http://images.amazon.com/images/P/0451203895.0...
4,A Bend in the Road,Nicholas Sparks,2001,http://images.amazon.com/images/P/0446527785.0...
...,...,...,...,...
701,Year of Wonders,Geraldine Brooks,2002,http://images.amazon.com/images/P/0142001430.0...
702,You Belong To Me,Mary Higgins Clark,1999,http://images.amazon.com/images/P/0671004549.0...
703,Zen and the Art of Motorcycle Maintenance: An ...,ROBERT PIRSIG,1984,http://images.amazon.com/images/P/0553277472.0...
704,Zoya,Danielle Steel,1989,http://images.amazon.com/images/P/0440203856.0...


In [36]:
similarity_scores.shape

NameError: name 'similarity_scores' is not defined

In [37]:
popular_df.to_csv('C:\\Users\\Dell\\Desktop\\book Recomender\pop.csv',index=True)

In [38]:
new = pd.read_csv('pop.csv')

In [39]:
new

,Unnamed: 0,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
1,3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
2,5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
3,9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
4,13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
5,16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
6,17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
7,26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
8,28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
9,39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [40]:
popular_df

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [41]:
new

,Unnamed: 0,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
1,3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
2,5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
3,9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
4,13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
5,16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
6,17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
7,26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
8,28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
9,39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [42]:
del new[index=0]

SyntaxError: invalid syntax (<ipython-input-42-20cc0848b4ff>, line 1)

In [43]:
new

,Unnamed: 0,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
1,3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
2,5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
3,9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
4,13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
5,16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
6,17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
7,26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
8,28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
9,39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [44]:
new = new.drop(new.columns[[0]],axis = 1)

In [45]:
new

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
1,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
2,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
3,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
4,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
5,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
6,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
7,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
8,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
9,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [46]:
new.to_csv('C:\\Users\\Dell\\Desktop\\book Recomender\pop.csv',index=False)

In [47]:
hel = pd.read_csv('pop.csv')

In [48]:
hel

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
1,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
2,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
3,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
4,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
5,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
6,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
7,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
8,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
9,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


In [49]:
h = pd.read_csv('new.csv')

In [500]:
h

,Book-Title,Book-Author,Year-Of-Publication,Image-URL-M
0,1984,George Orwell,1990,http://images.amazon.com/images/P/0451524934.0...
1,1st to Die: A Novel,James Patterson,2002,http://images.amazon.com/images/P/0446610038.0...
2,2nd Chance,James Patterson,2002,http://images.amazon.com/images/P/0316693200.0...
3,4 Blondes,Candace Bushnell,2002,http://images.amazon.com/images/P/0451203895.0...
4,A Bend in the Road,Nicholas Sparks,2001,http://images.amazon.com/images/P/0446527785.0...
...,...,...,...,...
701,Year of Wonders,Geraldine Brooks,2002,http://images.amazon.com/images/P/0142001430.0...
702,You Belong To Me,Mary Higgins Clark,1999,http://images.amazon.com/images/P/0671004549.0...
703,Zen and the Art of Motorcycle Maintenance: An ...,ROBERT PIRSIG,1984,http://images.amazon.com/images/P/0553277472.0...
704,Zoya,Danielle Steel,1989,http://images.amazon.com/images/P/0440203856.0...


In [501]:
abc = h.merge(avg_rating_df,on='Book-Title').drop_duplicates('Book-Title')
main_data = abc.merge(num_rating_df,on='Book-Title').drop_duplicates('Book-Title')

In [469]:
similarity_scores.shape

(706, 706)

In [470]:
similarity_scores

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [503]:
main_data['Image-URL-M']


0      http://images.amazon.com/images/P/0451524934.0...
1      http://images.amazon.com/images/P/0446610038.0...
2      http://images.amazon.com/images/P/0316693200.0...
3      http://images.amazon.com/images/P/0451203895.0...
4      http://images.amazon.com/images/P/0446527785.0...
                             ...                        
701    http://images.amazon.com/images/P/0142001430.0...
702    http://images.amazon.com/images/P/0671004549.0...
703    http://images.amazon.com/images/P/0553277472.0...
704    http://images.amazon.com/images/P/0440203856.0...
705    http://images.amazon.com/images/P/0449003787.0...
Name: Image-URL-M, Length: 706, dtype: object

In [473]:
# main_data.hide_index()
blankIndex=[''] * len(main_data)
main_data.index=blankIndex
main_data

,Book-Title,Book-Author,...,avg_ratings,num_ratings
,1984,George Orwell,...,4.454225,284
,1st to Die: A Novel,James Patterson,...,3.575639,509
,2nd Chance,James Patterson,...,3.269663,356
,4 Blondes,Candace Bushnell,...,1.947020,151
,A Bend in the Road,Nicholas Sparks,...,3.364162,346
...,...,...,...,...,...
,Year of Wonders,Geraldine Brooks,...,4.331361,169
,You Belong To Me,Mary Higgins Clark,...,2.731707,123
,Zen and the Art of Motorcycle Maintenance: An ...,ROBERT PIRSIG,...,3.147619,210
,Zoya,Danielle Steel,...,1.466019,103


In [505]:
# Book_Title = main_data['Book-Title']
# Book_Author = main_data['Book-Author']
# avg_rating = main_data['avg_ratings']
# num_rating = main_data['num_ratings']
img = main_data['Image-URL-M']

In [488]:
type(main_data)

pandas.core.frame.DataFrame

# how to convert a dataframe column to list so that we can import them as a json but after convert into list we have to import them by this method

json.dump(NameOfVariable, codecs.open('file path', 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4) 


In [506]:
# Book_Title = main_data["Book-Title"].values.tolist()
# print(col_list)
# Book_Title

img = main_data['Image-URL-M'].values.tolist()

In [493]:
# Book_Title.style.hide_index()
# Book_Title
Book_Author = main_data["Book-Author"].values.tolist()
avg_rating  = main_data["avg_ratings"].values.tolist()
num_rating = main_data["num_ratings"].values.tolist()

In [496]:
Book_Author
num_rating
avg_rating

[284,
 509,
 356,
 151,
 346,
 236,
 265,
 156,
 116,
 169,
 99,
 302,
 285,
 192,
 128,
 174,
 327,
 838,
 149,
 413,
 135,
 210,
 549,
 164,
 378,
 273,
 158,
 192,
 150,
 279,
 262,
 158,
 121,
 126,
 332,
 187,
 258,
 184,
 147,
 304,
 302,
 326,
 222,
 170,
 670,
 118,
 167,
 233,
 142,
 111,
 160,
 90,
 248,
 220,
 230,
 181,
 247,
 276,
 195,
 284,
 167,
 126,
 229,
 178,
 350,
 191,
 180,
 89,
 182,
 222,
 95,
 128,
 106,
 207,
 360,
 263,
 189,
 201,
 128,
 137,
 135,
 165,
 226,
 155,
 239,
 109,
 131,
 815,
 224,
 171,
 177,
 110,
 193,
 106,
 214,
 140,
 198,
 120,
 287,
 128,
 129,
 150,
 205,
 139,
 344,
 165,
 192,
 145,
 128,
 249,
 148,
 194,
 311,
 202,
 90,
 104,
 350,
 228,
 299,
 177,
 125,
 183,
 204,
 265,
 118,
 199,
 135,
 140,
 176,
 131,
 247,
 165,
 258,
 127,
 90,
 87,
 142,
 130,
 98,
 149,
 150,
 262,
 85,
 116,
 180,
 170,
 85,
 136,
 293,
 318,
 740,
 275,
 169,
 228,
 118,
 127,
 151,
 91,
 481,
 170,
 187,
 101,
 178,
 203,
 111,
 155,
 160,
 116,
 20

In [ ]:
col_list = df["Courses"].values.tolist()
print(col_list)

In [ ]:
hello.to_csv('C:\\Users\\Dell\\Desktop\\book Recomender\\new.csv',index=True)

In [491]:
Book_Title.to_json('C:\\Users\\Dell\\Desktop\\Project\\json\\recomender\\title.json',index=False)
# Book_Title
# Book_Title.list()

AttributeError: 'list' object has no attribute 'to_json'

In [507]:
# avg_rating.to_json('C:\\Users\\Dell\\Desktop\\Project\\json\\recomender\\avgRating.json',index=False)
# Book_Author.to_json('C:\\Users\\Dell\\Desktop\\Project\\json\\recomender\\author.json',index=False)
# num_rating.to_json('C:\\Users\\Dell\\Desktop\\Project\\json\\recomender\\numRating.json',index=False)
# similarity_scores.to_json('C:\\Users\\Dell\\Desktop\\Project\\json\\recomender\\similarity.json',index=False)

# Book_Author
# num_rating
# avg_rating

json.dump(img, codecs.open('C:\\Users\\Dell\\Desktop\\Project\\json\\recomender\\image.json', 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4) 



In [448]:
type(similarity_scores)

numpy.ndarray

In [451]:
bca = similarity_scores.tolist()